<a href="https://colab.research.google.com/github/unmtransinfo/Replicase/blob/master/Replicase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chembl_webresource_client
!pip install molvs rdkit

In [ ]:
# Mounting the Google Drive

import os, sys
from google.colab import drive

drive.mount('/content/drive', force_remount = True)
sys.path.insert(0, '/content/drive/My Drive/Replicase/')

# defining the file path
data_filepath = "/content/drive/My Drive/Replicase/data/"
python_dir_path = "/content/drive/MyDrive/Replicase/python-files/"

In [ ]:
sys.path.insert(1, python_dir_path)

In [ ]:
PRE_PROCESSED_REPLICASE_DATA_FILE = "replicase_data_preprocessed.csv"
PRE_PROCESSED_3CLPRO_DATA_FILE = "3cl-pro_data_preprocessed.csv"
STD_REPLICASE_DATA_FILE = "Replicase_stand_smi_data.csv"
STD_3CLPRO_DATA_FILE = "3cl-pro_stand_smi_data.csv"

# If this code is run on local computer, change the path to match your dir path
pre_process_replicase_data_path = data_filepath + PRE_PROCESSED_REPLICASE_DATA_FILE
pre_process_3clpro_data_path = data_filepath + PRE_PROCESSED_3CLPRO_DATA_FILE
std_replicase_data_path = data_filepath + STD_REPLICASE_DATA_FILE
std_3clpro_data_path = data_filepath + STD_3CLPRO_DATA_FILE

In [ ]:
# Other imports
import tempfile

import pandas as pd
import numpy as np
import molvs

from sklearn.impute import SimpleImputer

from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit.Chem import AllChem
from rdkit.Chem import PandasTools
from rdkit.Chem import Draw
from rdkit.Chem import MolStandardize
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem import MACCSkeys
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from tqdm import tqdm
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.drawOptions.comicMode=True
import rdkit

print(rdkit.__version__)

In [ ]:
# Importing python files to use internal functions
# These files are for data preprocessing and preparation.

from preprocess import main
from standardization_process import standard_preprocess

In [ ]:
# Check if pre processed files are present
'''
 Only std_replicase_data and std_3clpro_data are required after this code block
 If standard data already exists, assign the variables directly
'''

if not os.path.exists(pre_process_replicase_data_path) or not os.path.exists(pre_process_3clpro_data_path):
  pre_replicase_data, pre_3clpro_data = main()

  pre_replicase_data.to_csv(pre_process_replicase_data_path)
  pre_3clpro_data.to_csv(pre_process_3clpro_data_path)

if not os.path.exists(std_replicase_data_path):
  pre_replicase_data = pd.read_csv(pre_process_replicase_data_path)
  std_replicase_data = standard_preprocess(pre_replicase_data)
  std_replicase_data.to_csv(std_replicase_data_path)
else:
  std_replicase_data = pd.read_csv(std_replicase_data_path)

if not os.path.exists(std_3clpro_data_path):
  pre_3clpro_data = pd.read_csv(pre_process_3clpro_data_path)
  std_3clpro_data = standard_preprocess(pre_3clpro_data)
  std_3clpro_data.to_csv(std_3clpro_data_path)
else:
  std_3clpro_data = pd.read_csv(std_3clpro_data_path)


In [ ]:
std_replicase_data.head(5)
# std_3clpro_data.head(5)

In [ ]:
STANDARDIZED_SMILES = "standardized_smiles"
BIOACTIVITY_CLASS = "bioactivity_class"
PCHEMBL_VALUE = "pchembl_value"
CANONICAL_SMILES = "canonical_smiles"
MOLECULE_CHEMBL_ID = "molecule_chembl_id"
STANDARDIZED_MOLECULE = "standardized_molecule"

In [ ]:
REPLICASE_SDF_FILE = "replicase_temp.sdf"
_3CLPRO_SDF_FILE = "3clpro_temp.sdf"

In [ ]:
# Feature Generation

# Generate standardized molecules from standardized smiles
def generate_std_molecule(std_smiles):
  std_mols = []
  std_smiles = std_smiles.tolist()

  for smi in std_smiles:
    std_mol = Chem.MolFromSmiles(smi)
    std_mols.append(std_mol)

  return pd.DataFrame(std_mols)
  # df[STANDARDIZED_MOLECULE] = pd.DataFrame(std_mols)

# Generate Maccs keys from standardized molecule chemical fingerprints for substructure search
def generate_MACCSfpts(std_mols):
    maccs_fpts = []

    for mol in tqdm(std_mols):
        mkeyfpts = MACCSkeys.GenMACCSKeys(mol)
        maccs_fpts.append(mkeyfpts)

    return np.array(maccs_fpts)

# Generate path based fingerprints - rdkfingerprints, Daylight-like from standardized molecule
def generate_RDKfpts(std_mols):
    RDK_fpts = []

    for mol in tqdm(std_mols):
        rdkfpts = AllChem.RDKFingerprint(mol, maxPath=5, fpSize=2048, nBitsPerHash=2 )
        RDK_fpts.append(rdkfpts)

    return np.array(RDK_fpts)

# Generate toplogical fingerprints - atom pair from standardized molecule
def generate_APfpts(std_mols):
    AP_fpts = []

    for mol in tqdm(std_mols):
        apfpts = rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(mol, nBits=2048)
        AP_fpts.append(apfpts)

    return np.array(AP_fpts)

# Generate toplogical fingerprints - topological torsion from standardized molecule
def generate_TTfpts(std_mols):
    TT_fpts = []

    for mol in tqdm(std_mols):
        ttfpts = rdMolDescriptors.GetHashedTopologicalTorsionFingerprintAsBitVect(mol, nBits=2048)
        TT_fpts.append(ttfpts)

    return np.array(TT_fpts)

# Generate extended connectivity finger prints -  FINGERPRINTS from standardized molecule
def generate_Morganfpts(std_mols, radius):
    MORGAN_fpts = []

    for mol in tqdm(std_mols):
        morganfpts = AllChem.GetMorganFingerprintAsBitVect(mol,radius, nBits=2048)
        MORGAN_fpts.append(morganfpts)

    return np.array(MORGAN_fpts)

# Generate extended connectivity finger prints - FEATURE CONNECTIVITY FINGERPRINTS from standardized molecule
def generate_Featurefpts(std_mols, radius):
    FEATURE_fpts = []
    for mol in tqdm(std_mols):
        featurefpts = AllChem.GetMorganFingerprintAsBitVect(mol,radius, useFeatures=True, nBits=2048)
        FEATURE_fpts.append(featurefpts)
    return np.array(FEATURE_fpts)

In [ ]:
# Calculate the RDKit descriptors from standardized smiles
def RDKit_descriptors(smiles):
    mols = [Chem.MolFromSmiles(i) for i in smiles]
    calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
    desc_names = calc.GetDescriptorNames()
    mol_descriptors = []

    for mol in mols:
        mol = Chem.AddHs(mol)
        descriptors = calc.CalcDescriptors(mol)
        mol_descriptors.append(descriptors)

    return mol_descriptors, desc_names

# Apply mean imputation to rows with missing values
# def apply_mean(df):
#   mean_imputer = SimpleImputer(strategy='mean')
#   return mean_imputer.fit_transform(df)
mean_imputer = SimpleImputer(strategy='mean')

In [ ]:
# Convert the standardized smiles to an sdf file and generate file path
def toSDF(smiles_list, filename = "temp.sdf"):
    # temp_dir = tempfile.mkdtemp(dir = ".")
    w = Chem.SDWriter(filename)

    for smiles in smiles_list:
      mol = Chem.MolFromSmiles(smiles)
      if mol is not None:
          AllChem.Compute2DCoords(mol)
          w.write(mol)

    w.close()

    sdf_path = filename
    return sdf_path

In [ ]:
## Generate all fingerprints/properties for replicase

std_replicase_data[STANDARDIZED_MOLECULE] = generate_std_molecule(std_replicase_data[STANDARDIZED_SMILES])
replicase_std_mols = std_replicase_data[STANDARDIZED_MOLECULE]

# Generate MACCS keys
replicase_macc_keys = generate_MACCSfpts(replicase_std_mols)
replicase_macc_keys = pd.DataFrame(replicase_macc_keys, columns=['Maccs_{}'.format(i + 1)
                                  for i in range(replicase_macc_keys.shape[1])])

# Generate RDK fingerprints
replicase_rdk_fpts = generate_RDKfpts(replicase_std_mols)
replicase_rdk_fpts = pd.DataFrame(replicase_rdk_fpts, columns = ['RDK_{}'.format(i + 1) for i in range(replicase_rdk_fpts.shape[1])])

# Generate Atom Pair fingerprints
replicase_ap_fpts = generate_APfpts(replicase_std_mols)
replicase_ap_fpts = pd.DataFrame(replicase_ap_fpts, columns = ["AP_{}".format(i + 1) for i in range(replicase_ap_fpts.shape[1])])

# Generate Topological torsion fingerprints
replicase_tt_fpts = generate_TTfpts(replicase_std_mols)
replicase_tt_fpts = pd.DataFrame(replicase_tt_fpts, columns = ["TT_{}".format(i + 1) for i in range(replicase_tt_fpts.shape[1])])

# Generate Morgan fingerprints
replicase_morgan_fpts = generate_Morganfpts(replicase_std_mols, 2)
replicase_morgan_fpts = pd.DataFrame(replicase_morgan_fpts, columns = ["Morgan_{}".format(i + 1) for i in range(replicase_morgan_fpts.shape[1])])

# Generate Feature connectivity fingerprints
replicase_fc_fpts = generate_Featurefpts(replicase_std_mols, 2)
replicase_fc_fpts = pd.DataFrame(replicase_fc_fpts, columns = ["FC_{}".format(i) for i in range(replicase_fc_fpts.shape[1])])

# Generate RDK descriptors
descriptors, desc_names = RDKit_descriptors(std_replicase_data[STANDARDIZED_SMILES])
replicase_descriptors = pd.DataFrame(descriptors, columns=desc_names)

columns = replicase_descriptors.columns
replicase_descriptors = pd.DataFrame(mean_imputer.fit_transform(replicase_descriptors))
replicase_descriptors.columns = columns

# Saving standardized smiles to SDF file
replicase_sdf_path = toSDF(std_replicase_data[STANDARDIZED_SMILES], REPLICASE_SDF_FILE)
replicase_sdf_path

In [ ]:
## Generate all fingerprints/properties for 3clpro

std_3clpro_data[STANDARDIZED_MOLECULE] = generate_std_molecule(std_3clpro_data[STANDARDIZED_SMILES])
_3clpro_std_mols = std_3clpro_data[STANDARDIZED_MOLECULE]

# Generate MACCS keys
_3clpro_macc_keys = generate_MACCSfpts(_3clpro_std_mols)
_3clpro_macc_keys = pd.DataFrame(_3clpro_macc_keys, columns=['Maccs_{}'.format(i + 1)
                                  for i in range(_3clpro_macc_keys.shape[1])])

# Generate RDK fingerprints
_3clpro_rdk_fpts = generate_RDKfpts(_3clpro_std_mols)
_3clpro_rdk_fpts = pd.DataFrame(_3clpro_rdk_fpts, columns = ['RDK_{}'.format(i + 1) for i in range(_3clpro_rdk_fpts.shape[1])])

# Generate Atom Pair fingerprints
_3clpro_ap_fpts = generate_APfpts(_3clpro_std_mols)
_3clpro_ap_fpts = pd.DataFrame(_3clpro_ap_fpts, columns = ["AP_{}".format(i + 1) for i in range(_3clpro_ap_fpts.shape[1])])

# Generate Topological torsion fingerprints
_3clpro_tt_fpts = generate_TTfpts(_3clpro_std_mols)
_3clpro_tt_fpts = pd.DataFrame(_3clpro_tt_fpts, columns = ["TT_{}".format(i + 1) for i in range(_3clpro_tt_fpts.shape[1])])

# Generate Morgan fingerprints
_3clpro_morgan_fpts = generate_Morganfpts(_3clpro_std_mols, 2)
_3clpro_morgan_fpts = pd.DataFrame(_3clpro_morgan_fpts, columns = ["Morgan_{}".format(i + 1) for i in range(_3clpro_morgan_fpts.shape[1])])

# Generate Feature connectivity fingerprints
_3clpro_fc_fpts = generate_Featurefpts(_3clpro_std_mols, 2)
_3clpro_fc_fpts = pd.DataFrame(_3clpro_fc_fpts, columns = ["FC_{}".format(i) for i in range(_3clpro_fc_fpts.shape[1])])

# Generate RDK descriptors
descriptors, desc_names = RDKit_descriptors(std_3clpro_data[STANDARDIZED_SMILES])
_3clpro_descriptors = pd.DataFrame(descriptors, columns=desc_names)

_3cl_columns = _3clpro_descriptors.columns
_3clpro_descriptors = pd.DataFrame(mean_imputer.transform(_3clpro_descriptors))
_3clpro_descriptors.columns = _3cl_columns

# Saving standardized smiles to SDF file
_3clpro_sdf_path = toSDF(std_3clpro_data[STANDARDIZED_SMILES], _3CLPRO_SDF_FILE)
_3clpro_sdf_path

In [ ]:
# Download and extract mayachemtools
!rm mayachemtools.zip

!wget http://www.mayachemtools.org/download/mayachemtools.zip
# !tar -xvjf mayachemtools.tar.bz2
!unzip -o mayachemtools.zip


In [ ]:
!perl mayachemtools/bin/TopologicalPharmacophoreAtomTripletsFingerprints.pl --AtomTripletsSetSizeToUse FixedSize -v ValuesString -o $REPLICASE_SDF_FILE -r replicase_tpatf
!perl mayachemtools/bin/TopologicalPharmacophoreAtomTripletsFingerprints.pl --AtomTripletsSetSizeToUse FixedSize -v ValuesString -o $_3CLPRO_SDF_FILE -r 3clpro_tpatf

In [ ]:
# Extract fingerprints into array from CSV
def extract_features(filepath):
  with open(filepath, 'r') as f:
    all_features = []

    for line in f.readlines():
        if "Cmpd" in line:
            line = line.split(';')[5].replace('"', '')
            features = [int(i) for i in line.split(" ")]
            all_features.append(features)

    return all_features

replicase_features_array = np.array(extract_features("replicase_tpatf.csv"))
replicase_tpatf = pd.DataFrame(replicase_features_array, columns = ["tpatf_{}".format(i + 1) for i in range(replicase_features_array.shape[1])])

_3clpro_features_array = np.array(extract_features("3clpro_tpatf.csv"))
_3clpro_tpatf = pd.DataFrame(_3clpro_features_array, columns = ["tpatf_{}".format(i + 1) for i in range(_3clpro_features_array.shape[1])])